# Import and View Data

In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
data = pd.read_csv("../data/Case3HistoricalPrices.csv")
data = data.drop(columns=['Unnamed: 0'])
data.head()

,S1,S2,S3,S4,S5,S6,S7,S8,B1,B2,...,B7,B8,C1,C2,C3,C4,C5,C6,C7,C8
0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,98.355850,98.307641,95.630621,98.296988,98.474008,95.663069,98.342292,98.416837,100.086605,99.685232,...,99.674428,100.131478,95.854652,95.571202,94.085760,93.956485,94.140624,94.058865,95.786959,94.331430
2,96.766305,97.785655,95.211317,97.876191,96.910785,95.299426,97.862363,96.850881,100.595828,99.614514,...,99.463006,100.594669,87.500105,86.517650,88.861746,89.205952,89.255488,89.203200,86.873750,89.644765
3,99.498201,97.762932,95.831525,97.979482,99.992847,95.890271,97.890721,99.523488,100.687198,99.425275,...,99.220046,100.639175,84.500885,83.306741,86.183137,86.617107,86.643905,86.512592,83.788306,87.068357
4,96.259221,92.942010,94.272932,92.958011,96.469517,94.318577,93.007052,96.278064,101.939638,100.099827,...,99.843072,101.823484,79.974708,78.434282,80.585967,81.404902,81.484117,81.131945,79.157682,81.778226


In [3]:
train_split = 252*5 #5 Years
train = data.iloc[:train_split].reset_index(drop=True)
test = data.iloc[train_split:].reset_index(drop=True)

# Method 1:

Solving the optimization problem of minimizing $w^T Vw$ such that $1^T w = 1$ produces closed form solution
$$w = \frac{1}{1^T V^{-1} r}V^{-1} r$$
Which is implemented below. Here, $w$ is the weights and $V$ is the covariance matrix.

Note that while this method scores a negative total points, if you change the testing code to test on "train.iterrows()", you get an overall positive score. It's most likely overfitting on the training data and thus doesn't perform as well.

In [4]:
ones = np.ones(24)
r = (train.iloc[-1] - train.iloc[0]).to_numpy()
V = np.cov(train.to_numpy().T)
V_inv = np.linalg.inv(V)
print(V.shape)

train_weights = (1/(ones.T@V_inv@r))*V_inv@r

def allocate(asset_prices):
    return train_weights

(24, 24)


### Results

In [5]:
# Annualized daily sharpe ratio
weights = np.zeros(len(data.columns))
prev_row = np.zeros(len(data.columns))
returns = []

for index, row in test.iterrows():
    if index == 0: # No returns on first day
        prev_row = row.to_numpy()
        new_weights = allocate(prev_row)
    else: # Add daily return
        asset_prices = row.to_numpy()
        yesterday = prev_row * weights
        today = asset_prices * new_weights
        transaction_fee = 30*sum(abs(new_weights - weights))
        daily_return = sum(today) - sum(yesterday) - transaction_fee
        returns.append(daily_return)
        # Update prev_row and weights
        prev_row = asset_prices
        weights = new_weights
        new_weights = allocate(asset_prices)

In [6]:
total = 0
for i in range(1,6):
    year_ret = np.array(returns[(i-1)*252: i*252])
    if year_ret.std():
        ratio = math.sqrt(252)*year_ret.mean()/year_ret.std()
    else: ratio = 0
    total += ratio
    print("Sharpe ratio for year ", i, ": ", ratio)
print("Total points: ", total)

Sharpe ratio for year  1 :  -0.9250038881248382
Sharpe ratio for year  2 :  -0.21461813972705493
Sharpe ratio for year  3 :  0.2766426256327067
Sharpe ratio for year  4 :  -0.4598482511489446
Sharpe ratio for year  5 :  -0.6097257647753382
Total points:  -1.9325534181434694


# Method 2

Rather than deciding on one set of weights to use for the entire testing set, we can recalculate the weights by updating returns with every new set of asset prices. The equations used are the same as above.

In [7]:
ones = np.ones(24)
V = np.cov(train.to_numpy().T)
V_inv = np.linalg.inv(V)

def allocate(asset_prices):
    r = (asset_prices - train.iloc[0]).to_numpy()
    train_weights = (1/(ones.T@V_inv@r))*V_inv@r
    return train_weights

### Results

In [8]:
# Annualized daily sharpe ratio
weights = np.zeros(len(data.columns))
prev_row = np.zeros(len(data.columns))
returns = []

for index, row in test.iterrows():
    if index == 0: # No returns on first day
        prev_row = row.to_numpy()
        new_weights = allocate(prev_row)
    else: # Add daily return
        asset_prices = row.to_numpy()
        yesterday = prev_row * weights
        today = asset_prices * new_weights
        transaction_fee = 30*sum(abs(new_weights - weights))
        daily_return = sum(today) - sum(yesterday) - transaction_fee
        returns.append(daily_return)
        # Update prev_row and weights
        prev_row = asset_prices
        weights = new_weights
        new_weights = allocate(asset_prices)

In [9]:
total = 0
for i in range(1,6):
    year_ret = np.array(returns[(i-1)*252: i*252])
    if year_ret.std():
        ratio = math.sqrt(252)*year_ret.mean()/year_ret.std()
    else: ratio = 0
    total += ratio
    print("Sharpe ratio for year ", i, ": ", ratio)
print("Total points: ", total)

Sharpe ratio for year  1 :  -3.0511031304893197
Sharpe ratio for year  2 :  -2.4886495631390946
Sharpe ratio for year  3 :  -3.0341056099356836
Sharpe ratio for year  4 :  -1.9231045900767576
Sharpe ratio for year  5 :  -3.8116261891080656
Total points:  -14.30858908274892


# Method 3

This time, we recalculate the weights *and covariance matrix* with every new set of asset prices. The equations used are the same as above.

In [10]:
ones = np.ones(24)
prices_so_far = train.to_numpy()

def allocate(asset_prices):
    global prices_so_far
    prices_so_far = np.vstack((prices_so_far, asset_prices))
    V = np.cov(prices_so_far.T)
    V_inv = np.linalg.inv(V)
    r = (asset_prices - train.iloc[0]).to_numpy()
    train_weights = (1/(ones.T@V_inv@r))*V_inv@r
    return train_weights

### Results

In [11]:
# Annualized daily sharpe ratio
weights = np.zeros(len(data.columns))
prev_row = np.zeros(len(data.columns))
returns = []

for index, row in test.iterrows():
    if index == 0: # No returns on first day
        prev_row = row.to_numpy()
        new_weights = allocate(prev_row)
    else: # Add daily return
        asset_prices = row.to_numpy()
        yesterday = prev_row * weights
        today = asset_prices * new_weights
        transaction_fee = 30*sum(abs(new_weights - weights))
        daily_return = sum(today) - sum(yesterday) - transaction_fee
        returns.append(daily_return)
        # Update prev_row and weights
        prev_row = asset_prices
        weights = new_weights
        new_weights = allocate(asset_prices)

In [12]:
total = 0
for i in range(1,6):
    year_ret = np.array(returns[(i-1)*252: i*252])
    if year_ret.std():
        ratio = math.sqrt(252)*year_ret.mean()/year_ret.std()
    else: ratio = 0
    total += ratio
    print("Sharpe ratio for year ", i, ": ", ratio)
print("Total points: ", total)

Sharpe ratio for year  1 :  -3.40105680603709
Sharpe ratio for year  2 :  -3.235793039874136
Sharpe ratio for year  3 :  -6.723090787544591
Sharpe ratio for year  4 :  -2.9920922622292268
Sharpe ratio for year  5 :  -3.247593931315782
Total points:  -19.599626827000826
